In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import datetime as dt


import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_code(symbol):
    krx = pd.read_csv('./krx_code.csv',encoding='utf-8')
    krx = krx.set_index('한글 종목약명')
    try:
        code = krx.at[symbol,'단축코드']
        return code
    except:
        print('종목명을 다시 확인해주세요.')
        return 0

def get_comment_csv(symbol,page,year,month,day):   
    code = get_code(symbol)
    date_list = [] # 날짜
    comment_list = [] # 댓글
    view_list = [] # 조회수
    good_list = [] # 좋아요
    bad_list = [] # 싫어요
    flag = 0
    for i in range(1,page+1):
        url = f'https://finance.naver.com/item/board.naver?code={code}&page={i}'
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 Edg/100.0.1185.50'}
        res = requests.get(url, headers = headers)
        bs = BeautifulSoup(res.text, 'html.parser')
        for j in range(20):
            try:
                root = bs.find('div',{'class':'section inner_sub'}).find_all('tr',{'onmouseover':'mouseOver(this)'})[j].text.split('\n')
                
                date_list.append(root[1].replace('.','-'))
                
                if len(root) == 14: # 답글
                    comment_list.append('답글:'+root[4])
                    view_list.append(root[10])
                    good_list.append(root[11])
                    bad_list.append(root[12])          
                elif len(root) == 13: # 기본
                    comment_list.append(root[3])
                    view_list.append(root[9])
                    good_list.append(root[10])
                    bad_list.append(root[11])
                else: # 에러
                    comment_list.append('error')
                    view_list.append(0)
                    good_list.append(0)
                    bad_list.append(0)   
            except:
                break
#                 date_list.append('error')
#                 comment_list.append('error')
#                 view_list.append(0)
#                 good_list.append(0)
#                 bad_list.append(0)  
            tp = [int(j) for j in root[1].split()[0].split('.')]
            if dt.datetime(tp[0],tp[1],tp[2]) < dt.datetime(year,month,day):
                flag = 1
                break
        if flag == 1:
            break
        print(f'\r{i}페이지 크롤링 완료.',end='')
        
#         for i in date_list:
#             tp = [int(j) for j in i.split()[0].split('-')]
#             if dt.datetime(tp[0],tp[1],tp[2]) < dt.datetime(year,month,day):
#                 flag = 1
#                 break
#         if flag == 1:
#             break
        
    df = pd.DataFrame()
    df['날짜'] = date_list
    df['댓글'] = comment_list
    df['조회수'] = view_list
    df['좋아요'] = good_list
    df['싫어요'] = bad_list
    return df

In [58]:
pd.read_excel('./KODEX_KTOP_30_20220629.xlsx',header=2).drop(0,axis=0)

,번호,종목명,ISIN,종목코드,수량,비중(%),평가금액(원),현재가(원),등락(원)
1,2,삼성전자,KR7005930003,005930,6110,0.218129,362934000,58000,-1400
2,3,NAVER,KR7035420009,035420,611,0.092723,154277500,246500,-6000
3,4,삼성SDI,KR7006400006,006400,244,0.085202,141764000,570000,-11000
4,5,LG화학,KR7051910008,051910,244,0.083589,139080000,530000,-40000
5,6,카카오,KR7035720002,035720,1222,0.052586,87495200,70800,-800
6,7,유한양행,KR7000100008,000100,1222,0.041202,68554200,55700,-400
7,8,SK텔레콤,KR7017670001,017670,1222,0.040027,66599000,53000,-1500
8,9,POSCO홀딩스,KR7005490008,005490,244,0.035635,59292000,234000,-9000
9,10,현대모비스,KR7012330007,012330,244,0.030283,50386000,199000,-7500
10,11,SK이노베이션,KR7096770003,096770,244,0.029549,49166000,200000,-1500


In [8]:
ktop30_company = pd.read_excel('./KODEX_KTOP_30_20220629.xlsx',header=2).drop(0,axis=0)['종목명']

In [36]:
print(list(ktop30_company)[0:3])
print(list(ktop30_company)[10:14])

['삼성전자', 'NAVER', '삼성SDI']
['삼성화재', '롯데케미칼', '현대차', '셀트리온']


In [42]:
list(ktop30_company)[0]

'삼성전자'

In [5]:
df = get_comment_csv('미래에셋증권',500,2022,6,1) # sample


17페이지 크롤링 완료.

In [3]:
df = get_comment_csv('미래에셋증권',15,2022,6,15) # sample


11페이지 크롤링 완료.

In [6]:
df

,날짜,댓글,조회수,좋아요,싫어요
0,2022-07-09 21:27,미래에셋 버리고 타증권사 이동 추천,6,0,0
1,2022-07-09 16:05,오래 사세요,32,0,0
2,2022-07-08 23:40,본사 폭파시켜야,105,3,0
3,2022-07-08 18:37,아베가 아니라,128,3,1
4,2022-07-08 17:44,횡령 배임이 아니람 어서 왜 이 모양인지...,125,3,0
...,...,...,...,...,...
346,2022-06-02 09:07,531 색히야 공매치다 뒤진다,136,0,0
347,2022-06-02 07:40,"외국인348,831주 매수 연기금11...",261,0,3
348,2022-06-01 10:27,귀에 걸면 귀걸이 ?????,229,2,1
349,2022-06-01 22:17,답글:귀에 걸면 귀걸이 ?????,102,1,1


In [27]:
'2022-06-30' in df['날짜'][115]

True

In [26]:
df['날짜'][115]

'2022-06-30 14:00'

In [24]:
ktop30_company[10]

'SK이노베이션'

In [43]:
for i in range(3):
    df = get_comment_csv(list(ktop30_company)[i],100000,2020,6,1)
    #df = df[df['날짜']=='2022-07-01']
    df.to_csv(f'./src/2year_time/{list(ktop30_company)[i]}_2year.csv')

3048페이지 크롤링 완료..

In [23]:
#df.to_csv(f'./src/2year_time/현대차_year.csv')

In [18]:
pd.read_csv('./src/year_time/삼성전자_year.csv')

,Unnamed: 0,날짜,댓글,조회수,좋아요,싫어요
0,0,2022-07-09 21:44,이준석 몰래 이재명 만난 것으로 밝혀져,23,3,5
1,1,2022-07-09 21:44,갱산도 젖재명,8,1,1
2,2,2022-07-09 21:44,아베 총맞는 리얼 영상 떳네..ㄷㄷㄷ,12,0,0
3,3,2022-07-09 21:43,미국 경제지에서,13,7,0
4,4,2022-07-09 21:43,박근혜 vs 윤석열,10,2,0
...,...,...,...,...,...,...
568116,568116,2021-06-01 00:42,세계증시 /////////////////...,445,2,2
568117,568117,2021-06-01 00:31,중국 인도 ////////////////...,336,0,1
568118,568118,2021-06-01 00:19,떠라이 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...,364,2,1
568119,568119,2021-06-01 00:17,너희 석열이형이 보수라 생각하는거야?,396,0,1
